In [8]:
import pandas
# Read data, manipulate your data, select columns, basic stats, check empties.
data = pandas.read_csv("cleanedDataset.csv")
data


,Unnamed: 0,ids,bday,enrolldate,expgradate,Rank,Major,Gender,Athlete,Height,...,Math,Writing,State,LiveOnCampus,HowCommute,CommuteTime,SleepTime,StudyTime,Score,BMI
0,0,43783,1995-03-22,2000-01-01,2000-01-01,0.0,unknown,Male,0,22.052280,...,60.02,81.440000,In state,On Campus,Public Transit,27.0,7.0,1.0,311.200000,0.133728
1,1,20278,1995-01-01,2000-01-01,2000-01-01,0.0,Philosophy,Male,0,21.537168,...,70.19,73.270000,unknown,On Campus,Public Transit,27.0,5.0,2.0,318.160000,0.140201
2,2,20389,1994-12-31,2000-01-01,2000-01-01,0.0,unknown,Male,0,21.543264,...,71.20,84.240000,In state,unknown,Public Transit,27.0,8.0,7.0,339.050000,0.140122
3,3,22820,1994-12-01,2000-01-01,2000-01-01,0.0,business administration,Female,0,20.736082,...,55.89,73.160000,In state,On Campus,Public Transit,27.0,2.0,6.0,291.790000,0.151243
4,4,24559,1994-11-10,2000-01-01,2000-01-01,0.0,unknown,Female,1,20.552664,...,65.52,80.450000,Out of state,On Campus,Public Transit,27.0,7.0,3.0,305.880000,0.153954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,430,34021,1987-07-18,2011-08-01,2017-05-08,4.0,accounting,Female,0,20.736082,...,65.31,79.521683,In state,Off Campus,Walk,18.0,1.0,10.0,317.541683,0.151243
431,431,40697,1987-04-29,2011-08-01,2016-05-09,4.0,unknown,Male,0,20.878800,...,70.25,82.670000,In state,Off Campus,Car,26.0,6.0,15.0,330.800000,0.149182
432,432,34272,2000-01-01,2011-08-01,2016-05-09,4.0,nursing,Male,1,20.756880,...,58.16,81.870000,unknown,Off Campus,Car,29.0,6.0,10.0,306.840000,0.150940
433,433,33628,2000-01-01,2011-08-01,2017-05-08,4.0,Nursing,Female,1,21.113496,...,66.84,79.350000,In state,Off Campus,Walk,14.0,4.0,10.0,307.750000,0.145884


In [12]:
# 1.	Is there a statistically significant difference in average sleep time between students who smoke and those who don't?
#replacing smoking values with strings
data['Smoking'].replace({
    0:'Non Smoker',
    1:'Smoker',
    2:'Chain Smoker',
    3:'Unknown'
},inplace=True) #key: previous value, value: the new value

data.groupby('Smoking').size()

5.505313765182186

In [23]:
# 1.	Is there a statistically significant difference in average sleep time between students who smoke and those who don't?
import scipy.stats as stats

# Group data by smoking status and calculate sleep time means
smokers_sleep = data[data['Smoking'] == 'Non Smoker']['SleepTime'].mean()
non_smokers_sleep = data[data['Smoking'] == 'Smoker']['SleepTime'].mean()

# Perform t-test to compare means
tstat, pval = stats.ttest_ind(a=smokers_sleep, b=non_smokers_sleep)

# Print results with significance level (alpha = 0.05)
print(f"Average Sleep Time (Smokers): {smokers_sleep:.2f}") #.2f means in 2 decimal places
print(f"Average Sleep Time (Non-Smokers): {non_smokers_sleep:.2f}")
print(f"t-statistic: {tstat:.4f}, p-value: {pval:.4f}")# .4f means in 4 decimal places

if pval < 0.05:
  print("There is a statistically significant difference in sleep time between smokers and non-smokers.")
else:
  print("There is not sufficient evidence to conclude a difference in sleep time based on smoking status.")

Average Sleep Time (Smokers): 5.51
Average Sleep Time (Non-Smokers): 4.98
t-statistic: 1.2841, p-value: 0.2000
There is not sufficient evidence to conclude a difference in sleep time based on smoking status.


In [26]:
# 2.	Do athletes have a higher average writing score compared to non-athletes?
data['Athlete'].replace({
    0:'Yes',
    1:'No'
},inplace=True)

athletes_writing = data[data['Athlete'] == 'Yes']['Writing'].mean()
non_athletes_writing = data[data['Athlete'] == 'No']['Writing'].mean()

tstat, pval = stats.ttest_ind(data[data['Athlete'] == 'Yes']['Writing'], data[data['Athlete'] == 'No']['Writing'])

print(f"Average Writing Score (Athletes): {athletes_writing:.2f}")
print(f"Average Writing Score (Non-Athletes): {non_athletes_writing:.2f}")
print(f"t-statistic: {tstat:.4f}, p-value: {pval:.4f}") 



Average Writing Score (Athletes): 80.16
Average Writing Score (Non-Athletes): 78.64
t-statistic: 2.9748, p-value: 0.0031


In [28]:
# 3.	Is there a statistically significant difference in average exam scores (English, Reading, Math, Writing) across different states?
# Group data by 'State' and calculate means for each exam score
average_scores_by_state = data.groupby('State')[['English', 'Reading', 'Math', 'Writing']].mean()

# Perform ANOVA for each exam score using a loop
for col in average_scores_by_state.columns:
  fstat, pval = stats.f_oneway(*[list(average_scores_by_state[col]) for _, group in average_scores_by_state.groupby('State')])
  print(f"\nANOVA for {col} Scores:")
  print(f"F-statistic: {fstat:.4f}, p-value: {pval:.4f}")
  if pval < 0.05:
    print(f"There is a statistically significant difference in average {col} scores across states.")
  else:
    print(f"There is not sufficient evidence to conclude a difference in average {col} scores across states based on location.")


ANOVA for English Scores:
F-statistic: 0.0000, p-value: 1.0000
There is not sufficient evidence to conclude a difference in average English scores across states based on location.

ANOVA for Reading Scores:
F-statistic: 0.0000, p-value: 1.0000
There is not sufficient evidence to conclude a difference in average Reading scores across states based on location.

ANOVA for Math Scores:
F-statistic: 0.0000, p-value: 1.0000
There is not sufficient evidence to conclude a difference in average Math scores across states based on location.

ANOVA for Writing Scores:
F-statistic: 0.0000, p-value: 1.0000
There is not sufficient evidence to conclude a difference in average Writing scores across states based on location.


In [27]:
# 4.	Do students who live on campus have significantly different average English scores compared to commuting students?
live_on_campus_english = data[data['LiveOnCampus'] == 'On Campus']['English'].mean()
commute_english = data[data['LiveOnCampus'] == 'Off Campus']['English'].mean()

# Perform ANOVA F-test to compare means across groups
fstat, pval = stats.f_oneway(data[data['LiveOnCampus'] == 'On Campus']['English'], data[data['LiveOnCampus'] == 'Off Campus']['English'])

# Print results with significance level (alpha = 0.05)
print(f"Average English Score (Live On Campus): {live_on_campus_english:.2f}")
print(f"Average English Score (Commute): {commute_english:.2f}")
print(f"F-statistic: {fstat:.4f}, p-value: {pval:.4f}")

if pval < 0.05:
  print("There is a statistically significant difference in average English scores between students who live on campus and commuters.")
else:
  print("There is not sufficient evidence to conclude a difference in English scores based on living arrangements.")

Average English Score (Live On Campus): 82.21
Average English Score (Commute): 83.14
F-statistic: 1.9944, p-value: 0.1586
There is not sufficient evidence to conclude a difference in English scores based on living arrangements.


In [29]:
# 6.	Does being an athlete affect a student's average study time?
athlete_study_time = data[data['Athlete'] == 'Yes']['StudyTime']
non_athlete_study_time = data[data['Athlete'] == 'No']['StudyTime']

# Perform an independent two-sample t-test
t_statistic, p_value = stats.ttest_ind(athlete_study_time.dropna(), non_athlete_study_time.dropna(), nan_policy='omit')

# Interpret the results
print(f"T-Statistic: {t_statistic}")
print(f"P-Value: {p_value}")

alpha = 0.05
if p_value < alpha:
    print("There is a significant difference in average study time between athletes and non-athletes.")
else:
    print("There is no significant difference in average study time between athletes and non-athletes.")

T-Statistic: 0.12708524289790918
P-Value: 0.8989319256686695
There is no significant difference in average study time between athletes and non-athletes.


In [30]:
# Is there a significant difference in average sleep time between male and female students?
# Filter the data for male and female students
male_sleep_time = data[data['Gender'] == 'Male']['SleepTime']
female_sleep_time = data[data['Gender'] == 'Female']['SleepTime']

# Step 4: Perform an independent two-sample t-test
t_statistic, p_value = stats.ttest_ind(male_sleep_time.dropna(), female_sleep_time.dropna(), nan_policy='omit')

# Step 5: Interpret the results
print(f"T-Statistic: {t_statistic}")
print(f"P-Value: {p_value}")

# Determine significance
alpha = 0.05
if p_value < alpha:
    print("There is a significant difference in average sleep time between male and female students.")
else:
    print("There is no significant difference in average sleep time between male and female students.")

T-Statistic: 1.2682016446164621
P-Value: 0.20542185089633885
There is no significant difference in average sleep time between male and female students.
